# Task 8.2 Open QASM3

## Objective : Interoperate different versions of QASM3 with qiskit

Qiskit provides tools to export quantum circuits to QASM3 format and import QASM3 files back into Qiskit circuits.


### Exporting to QASM 3

#### Export Methods

* **`dump()`**: Export circuit to a file
* **`dumps()`**: Export circuit to an Open QASM3 string

In [ ]:
# Example export to file and string
from qiskit import QuantumCircuit
from qiskit.qasm3 import dump, dumps

# Create a simple circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Export to string
qasm_str = dumps(qc)
print("QASM3 String:\n")
print(qasm_str)

# Export to file
with open("test_circuit.qasm", "w") as f:
    dump(qc, f)

#### Exporter Class

It can be used to customize the export process.

**Parameters**:
* **`includes`**: List of files to include (default: `["stdgates.inc"]`)
* **`basis_gates`**: Limit to specific gate set for hardware compatibility
* **`disable_constants`**: Disable constant folding optimizations
* **`alias_classical_registers`**: Use aliases for classical registers
* **`allow_aliasing`**: Allow use of `let` statements for qubit aliasing
* **`indent`**: String used for indentation (default: "  ")
* **`experimental`**: Enable experimental features
* **`annotation_handlers`**: Custom handlers for circuit annotations

In [ ]:
# Example with Exporeter class
from qiskit import QuantumCircuit
from qiskit.qasm3 import Exporter

qc = QuantumCircuit(3)
qc.h(0)
qc.rx(1.57, 1)  # π/2 rotation

exporter = Exporter(
    # Don't include stdgates.inc
    includes=[],
    # Limit to specific gates
    basis_gates=["h", "rx"],
    # 4-space indentation
    indent="    ",
    allow_aliasing=True
)

# Export with custom exporter
qasm_str = exporter.dumps(qc)
print(qasm_str)


#### QASM3ExporterError

The exporter raises `QASM3ExporterError` when it encounters unsupported features.

In [ ]:
from qiskit import QuantumCircuit, qasm3
from qiskit.providers.fake_provider import GenericBackendV2

from qiskit.qasm3.exporter import QASM3ExporterError

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
# invalid use to test error handling
backend = GenericBackendV2(2)
exporter = qasm3.Exporter(basis_gates=backend.operation_names)

try:
    qasm_str = exporter.dumps(qc)
    print(f"Export successful: {qasm_str}")
except QASM3ExporterError as e:
    print(f"Export failed: {e}")

#### Experimental Features

Access experimental features through `ExperimentalFeatures`

### Importing from QASM 3

#### Import Methods

* **`load()`**: Import circuit from a file
* **`loads()`**: Import circuit from a string


**Note:**
package qiskit_qasm3_import must be installed. This can be done using
```pip install qiskit_qasm3_import```

In [ ]:
from qiskit.qasm3 import loads

# Example import from QASM3 string
qasm_code = """
OPENQASM 3.0;
include "stdgates.inc";

qubit[3] q;
bit[3] c;

h q[0];
cx q[0], q[1];

c[0] = measure q[0];
c[1] = measure q[1];
"""

# Import from string
circuit = loads(qasm_code)

circuit.draw('mpl')

In [ ]:
from qiskit.qasm3 import load

# Example Import from file
circuit = load("test_circuit.qasm")
circuit.draw('mpl')


#### QASM3ImoporterError

The importer raises `QASM3ImoporterError` when it encounters unsupported features.

In [ ]:
from qiskit.qasm3 import QASM3ImporterError

# Invalid QASM3 code
invalid_qasm = """
OPENQASM 3.0;
include "stdgates.inc";
qubit q;  
rh q; // unknown gate
"""

try:
    circuit = loads(invalid_qasm)
except QASM3ImporterError as e:
    print(f"Import failed: {e}")

#### Experimental Import Interface

Experimental Rust-based importer that is much faster 

##### Import Methods

* **`load_experimental()`**: Import circuit from a file
* **`loads_experimental()`**: Import circuit from a string

In [ ]:
from qiskit.qasm3 import loads_experimental

# Example import from QASM3 string
qasm_code = """
OPENQASM 3.0;
include "stdgates.inc";

qubit[3] q;
bit[3] c;

h q[0];
cx q[0], q[1];

c[0] = measure q[0];
c[1] = measure q[1];
"""

# Import from string
circuit = loads_experimental(qasm_code)

circuit.draw('mpl')

---

## Practice Questions

**1) When the Exporter class is used to customize QASM3 export, which parameter allows you to restrict the exported gates to a specific set for hardware compatibility?**

A) allow_aliasing

B) basis_gates

C) includes

D) disable_constants


***Answer:***
<Details>
B) basis_gates parameter limits the gate set to use on a specific target hardware
<br/>
</Details>

---

**2) What will be printed when the following code is executed??**

```
from qiskit.qasm3 import loads

qasm_string = """

OPENQASM 3.0;
include "stdgates.inc";

qubit[3] q;
bit[3] c;

h q[0];
cx q[0], q[1];

c[0] = measure q0;
c[1] = measure q1;
"""

try:
    circuit = loads(qasm_string)
    print("Circuit loaded successfully")
except Exception as e:
    print(f"Error: {type(e).__name__}")
```

A) Circuit loaded successfully

B) Error: QASM3ExporterError

C) Error: QASM3ImportError

D) Error: QASM3ImporterError


***Answer:***
<Details>
D) QASM3ImporterError will be thrown as measurment is done on non existant qubits q0 and q1
<br/>
</Details>

---